In [1]:
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import concurrent.futures

In [2]:
path="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Acc_w\\"
pathex=[]
temp = [f for f in listdir(path) if isfile(join(path, f))]
# temp.sort()
for i in range(len(temp)):
    st=temp[i]
    if(os.path.getsize(path+st)<4096):
        continue
#     if(temp[i][:6]=="b56522"):
#         pathex.append(temp[i])
    pathex.append(temp[i])
# print(pathex)
print(len(temp))
print(len(pathex))

4613
4571


In [3]:
# folder_n_no.ofvalues_no.offiles_keycolno.
propath="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Processed\\"
def savefile():
    df.to_csv (propath+"Acc_w_"+str(n)+"_4vuni_all_"+str(9)+".csv", index = False, header=True)

In [4]:
#uniform time and take only 4 values
# n -> number of lines in csv file
# actual -> the actual key pressed
# base -> round to base 5 (interval)
# minkeyd -> index of nearest key down time
# minkeyu -> index of nearest key up time
# keydti -> key down time start index from file name
# keyuti -> key up time start index from file name
def myround(x,base=5):
    return base*round(x/base)
n=30
actual=1
base=5
df=pd.read_csv(path+pathex[0])
for i in range(len(pathex)):
    try:
        dff=pd.read_csv(path+pathex[i])
        mini=10000
        minkeyd=1000
        minkeyu=1000
        keydti=pathex[i].rfind("_")+4
        keyuti=pathex[i].rfind("&")+1
        keydt=int(pathex[i][keydti:keyuti-1])
        keyut=int(pathex[i][keyuti:-4])
        dff1=dff.copy(deep=True)
        dff1.drop(dff1.index, inplace=True)
    #     print(dff1)
        mid=myround(keydt)
        start=mid-(12*5)
        end=mid+(11*5)
        t=start
#         print("before start : ",pathex[i])
        j=0
        while(j < len(dff.index)-1):
            if(dff['Time'][j]<0):
                break
            lb=abs(myround(dff['Time'][j])-t)
            ub=abs(myround(dff['Time'][j+1])-t)
            
            if(lb<ub):
    #             if(t>end):
    #                 break
                count+=1
                if(count>1):
                    count=0
                    j+=1
                    continue
#                 print("here : ",j)
                temp=pd.DataFrame({"Time":[t],"X":[dff['X'][j]],"Y":[dff['Y'][j]],"Z":[dff['Z'][j]]})
                dff1=dff1.append(temp,ignore_index=True)
                t+=5
            else:
                count=0
                j+=1
    #     print(dff1)
        dff=dff1
        dff=dff.head(n)

        mini=10000
        minkeyd=12
        minkeyu=12
        keydt=myround(int(pathex[i][keydti:keyuti-1]))
        keyut=myround(int(pathex[i][keyuti:-4]))
        for j in dff.index:
            t=abs(dff['Time'][j] - keydt)
            if(t < mini):
                mini=t
                minkeyd=j
        mini=10000
        for j in dff.index:
            t=abs(dff['Time'][j] - keyut)
            if(t < mini):
                mini=t
                minkeyu=j

        window=minkeyu-minkeyd
        
#         print("start : ",pathex[i])
        
        df2=dff.copy(deep=True)
        df2=df2.head(minkeyu)
        if(window<0):
            continue
        if(window==0):
            df2=df2.tail(1)
        else:
            df2=df2.tail(window)
        df2.reset_index(inplace=True)
        df2=df2.drop(columns="index")
        df2=df2.drop(columns="Time")

        dff.reset_index(inplace=True)
        dff=dff.drop(columns="index")
        dff=dff.drop(columns="Time")

        df1=dff.copy(deep=True)
        df1.loc[dff.index[0]]=dff.iloc[0]
        if(window==1 or window==0 or window==2):
            df1.loc[dff.index[1]]=df2.iloc[0]
        else:
            df1.loc[dff.index[1]]=df2.iloc[int(window/2)]
        df1.loc[dff.index[2]]=dff.iloc[-1]
        df1=df1.head(3)
        dff=df1

        df3=dff.stack().swaplevel()
        df3.index=df3.index.map('{0[0]}_{0[1]}'.format) 
        dff=df3.to_frame().T
#         print("here : ",pathex[i])
        ind=pathex[i].rfind("_")+3
        dff.insert(9, "key", int(pathex[i][ind]))
        if(i==0):
            df=dff
        else:
            df=df.append(dff,ignore_index = True)
    except Exception:
#         print("minkeyd : ",minkeyd)
#         print("minkeyu : ",minkeyu)
#         print("window : ",window)
        print("error : ",pathex[i])
#         traceback.print_exc()
        continue
# df

error :  bb7a16-02-2020_01_18_31_pm1254&287.csv


In [5]:
savefile()